# Training a Classifier on the *Salammbô* Dataset with Keras
Author: Pierre Nugues

We first need to import some modules

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import numpy as np

### Reading the dataset
We can read the data from a file with the svmlight format or directly create numpy arrays

In [2]:
X = np.array(
    [[35680, 2217], [42514, 2761], [15162, 990], [35298, 2274],
     [29800, 1865], [40255, 2606], [74532, 4805], [37464, 2396],
     [31030, 1993], [24843, 1627], [36172, 2375], [39552, 2560],
     [72545, 4597], [75352, 4871], [18031, 1119], [36961, 2503],
     [43621, 2992], [15694, 1042], [36231, 2487], [29945, 2014],
     [40588, 2805], [75255, 5062], [37709, 2643], [30899, 2126],
     [25486, 1784], [37497, 2641], [40398, 2766], [74105, 5047],
     [76725, 5312], [18317, 1215]
     ])

y = np.array(
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Scaling the Data
Scaling and normalizing are usually very significant with neural networks. We use sklean transformers. They consist of two main methods: `fit()` and `transform()`.

### Normalizing

In [3]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
normalizer.fit(X)
X_norm = normalizer.transform(X)
X_norm[:4]

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


array([[0.99807515, 0.06201605],
       [0.99789783, 0.06480679],
       [0.99787509, 0.06515607],
       [0.99793128, 0.06428964]])

### Standardizing

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(X_norm)
X_scaled = scaler.transform(X_norm)
X_scaled[:4]

array([[ 1.68336574, -1.7197772 ],
       [ 0.57376529, -0.56145427],
       [ 0.43143908, -0.41648279],
       [ 0.78308579, -0.77610221]])

## Creating a Model

We set a seed to have reproducible results

In [5]:
np.random.seed(1337)

We create a classifier equivalent to a logistic regression 

In [6]:
model = Sequential([
        Dense(1, input_dim=2, activation='sigmoid')
    ])

2022-08-23 18:52:45.886215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Or with one hidden layer

In [7]:
model2 = Sequential([
        Dense(10, input_dim=2, activation='relu'),
        # Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

To try the network with one hidden layer, set `complex` to true

In [8]:
complex = False
if complex == True:
    model = model2

## Fitting the Model

We compile and fit the model

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_scaled, y, epochs=30, batch_size=1, verbose=False)

### The weights

In [10]:
model.get_weights()

[array([[-1.4739954],
        [ 1.045873 ]], dtype=float32),
 array([0.02531876], dtype=float32)]

## Prediction

We compute the probabilities to belong to class 1 for all the training set

In [11]:
predicted_probs = model.predict(X_scaled, batch_size=1)
predicted_probs[:4]

30/30 [==============================] - 0s 617us/step


array([[0.01399966],
       [0.19660993],
       [0.25995383],
       [0.12557644]], dtype=float32)

We recompute it with matrices

In [12]:
from tensorflow.keras.activations import sigmoid, relu
if complex:
    print(sigmoid((relu(X_scaled@model.get_weights()[0] + model.get_weights()[1]))@model.get_weights()[2] + model.get_weights()[3])[:4])
else:
    print(sigmoid((X_scaled@model.get_weights()[0] + model.get_weights()[1]))[:4])

tf.Tensor(
[[0.01399967]
 [0.19660995]
 [0.25995382]
 [0.12557641]], shape=(4, 1), dtype=float64)


## Evaluation

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
def predict_class(preds):
    c = []
    for x in range(len(preds)):
        if(preds[x] >= 0.5):
            c += [1]
        else:
            c += [0]
    return np.array(c)

In [15]:
classes = predict_class(predicted_probs)
classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [16]:
accuracy_score(y, classes)

1.0

We computed the accuracy from the training set. This is not a good practice. We should use a dedicated test set instead.